In [24]:
import pandas as pd
import random
total_population = 4
class Teacher:
    def __init__(self, name):
        self.name = name
    def __str__(self): return str(self.name)
class Course:
    def __init__(self, code, name, section,teacher):
        self.code = code
        self.name = name
        self.teacher = teacher
        self.section = section
    def __str__(self): return self.name
class Section:
    def __init__(self, section):
        self.name = section
    def __str__(self): return str(self.name)
class ClassRoom:
    def __init__(self, code):
        self.code = code
class Duration:
    def __init__(self, time, day):
        self.time = time
        self.day = day
    def __str__(self): return (str(self.time) + ',' + str(self.day))
class Class:
    def __init__(self, id, course):
        self.id = id 
        self.course = course
        self.teacher = None
        self.section = None
        self.duration = None
        self.classroom = None
    def __str__(self):
        return str(self.course.name) +","+ str(self.section) +","+ str(self.teacher.name) +","+ str(self.classroom.code) +","+ str(self.duration.time)

In [25]:
class Information: #will contain and initialize the data
    section_ = ['A','B']
    teacher_ = ['Abdul Jalil', 'Salib', 'Numan', 'Hamza']
    room = ['A-001','A-002','A-003','A-004']
    day = ['monday','tuesday','wednesday','thursday','friday']
    #day = ['monday','tuesday','wednesday']
    period = [[830,930],[930,1030],[1030,1130],[1130,1230]]
    def __init__(self):
        self.classrooms = [ClassRoom(i) for i in self.room]
        self.section = [Section(i) for i in self.section_]
        self.teacher = [Teacher(i) for i in self.teacher_]
        self.duration = []
        for i in self.day:
            for j in self.period:self.duration.append(Duration(j,i))
        c1 = Course("1", "ITC", teacher = [self.teacher[0], self.teacher[1]], section = [self.section[0]])
        c2 = Course("2", "DIP", teacher = [self.teacher[2]], section =  [self.section[0], self.section[1]])
        c3 = Course("3", "OOP", teacher = [self.teacher[3], self.teacher[1]],  section = [self.section[0], self.section[1]])
        c4 = Course("4", "AI", teacher = [self.teacher[3]],  section = [self.section[0]])
        self.course = [c1, c2, c3, c4]
        self.Total_Class = 0
        for i in self.course:self.Total_Class += len(i.section)

In [26]:
class Schedule:
    def __init__(self):
        self.info_ = info
        self.classes = []
        self.total_conf = 0
        self.fitness_value = -1
        self.fit = False
        self.nmbclass = 0
    def fitnessValue(self):
        if self.fit == False :self.fitness_value= self.calculate_fitness();self.fit = True
        return self.fitness_value
    def calculate_fitness(self):
        self.total_conf, class_, teacher_ = 0, self.classes, []
        for i in class_:
            for j in class_:
                if ( str(i.duration) == str(j.duration) and i != j ):
                    if (i.classroom == j.classroom) or (i.teacher.name == j.teacher.name) or (i.section == j.section):
                        self.total_conf += 1
                        print('classroom, teacher, section conflict')
            teacher_.append([i.teacher.name, i.duration.time[0], i.duration.time[1], i.duration.day])
            if i.duration.time[0]<830 or i.duration.time[1] > 1700 or i.duration.time[0] > 1700:self.total_conf += 2;print('no class after 5 adn before 8:30') 
            if i.duration.day == 'saturday' or i.duration.day == 'sunday':self.total_conf += 2;print('no class on sunday and saturday')
            if i.duration.day == 'friday' and (i.duration.time[0]>=1300 or i.duration.time[1]<=1400):self.total_conf += 2;print('no class on friday between 1:00 t0 2:00')
        self.total_conf /= 2
        df_2 = pd.DataFrame(teacher_) 
        df_2 = df_2.sort_values(by=[df_2.columns[0], df_2.columns[2]])
        tech = df_2[df_2.columns[0]].unique()
        for i in tech:
            temp = df_2.loc[df_2[df_2.columns[0]] == i]
            day = temp[temp.columns[3]].unique()
            for j in day:
                temp1 = temp.loc[temp[temp.columns[3]] == j]
                strt, end, val = temp1.iat[0,1], temp1.iat[0,2], 0
                for k in range(0, int(temp1[0].count())):
                    if end == temp1.iat[k,1]:
                        val += 1
                        if val>1:
                            self.total_conf += 1
                            print('teacher consective classes',i, self.total_conf, j)
                            val = 0
                    else:val = 0
                    strt, end = temp1.iat[k,1], temp1.iat[k,2]
        return 1/float(1+self.total_conf)
    def initalize(self):
        course_ = self.info_.course
        for i in course_:
            for j in i.section:
                AddClass = Class(self.nmbclass, i)
                AddClass.section = j
                AddClass.duration = info.duration[random.randrange(0, len(info.duration))]
                AddClass.teacher = i.teacher[random.randrange(0, len(i.teacher))]
                AddClass.classroom = info.classrooms[random.randrange(0, len(info.classrooms))]
                self.nmbclass += 1
                self.classes.append(AddClass)
        return self
    def add_class(self, data_sample):
        course_ = self.info_.course
        for i in course_:
            if i.name == data_sample[1]:
                teach = [x.name for x in i.teacher]
                sec = [x.name for x in i.section]
                if data_sample[0] not in teach:i.teacher.append(Teacher(data_sample[0]))
                if data_sample[2] not in sec:i.section.append(Section(data_sample[2]))
        if data_sample[1] not in [i.name for i in course_]:
            course_.append(Course(str(len(course_)),name = data_sample[1], section=[Section(data_sample[2])], teacher=[Teacher(data_sample[0])]))
    def search(self, name, course, sec, dur, croom):
        for i in self.classes:
            if  name in [x.name for x in i.course.teacher]  and i.section.name == sec and i.course.name == course:
                i.duration = dur
                i.classroom = croom
                i.teacher = Teacher(name)
    def __str__(self):
        returnValue = ""
        for i in self.classes:returnValue += str(i) + "|"
        return returnValue

In [27]:
class Population:
    def __init__(self, size):
        self.size = size
        self.schedule = []
        for _ in range(size):
            data_sample = [ ['Abdul Jalil', 'ITC', 'B', ['monday',830,930], ['A-001','A-002','A-003']],
                            ['Abdul Jalil', 'ITC', 'C', ['monday',930,1030], ['A-001','A-002','A-003']],
                            ['Abdul Jalil', 'ITC', 'D', ['monday',1030,1130], ['A-001','A-002','A-003']],
                            ['Asad', 'HCI', 'A', ['monday',730,830], ['A-001','A-002','A-003']],
                            ['Asad', 'HCI', 'B', ['monday',1700,1830], ['A-001','A-002','A-003']],
                            ['Salib', 'DLD', 'A', ['friday',1300,1400], ['A-001','A-002','A-003']],
                            ['Moosa', 'DLD', 'B', ['saturday',1300,1400], ['A-001','A-002','A-003']],
                            ]
            x = Schedule()
            for i in data_sample:x.add_class(i)
            self.schedule.append(x.initalize())
            for i in data_sample:
                dur = Duration(time=[i[3][1],i[3][2]], day=i[3][0])
                croom = ClassRoom(i[4][random.randrange(0, len(i[4]))])
                x.search(i[0], i[1], i[2], dur, croom)

In [28]:
class GenAlgo:
    def __init__(self, population):
        self.population = population
    def start(self):
        new_pop = Population(0)
        for _ in range(0, total_population):new_pop.schedule.append(self.mutation(self.crossover(self.selection(self.population))))
        return new_pop
    def selection(self, pop):
        schedule = pop.schedule
        new_pop = Population(0)
        for _ in range(0,2):new_pop.schedule.append(schedule[random.randrange(0, total_population-1)])
        return new_pop
    def crossover(self, pop):
        newSchedule = Schedule().initalize()
        sch1 = pop.schedule[0]
        sch2 = pop.schedule[1]
        for idx in range(0, len(newSchedule.classes)):
            if random.random() < 0.5:newSchedule.classes[idx] = sch2.classes[idx]
            else:newSchedule.classes[idx] = sch1.classes[idx]
        return newSchedule
    def mutation(self, sched):
        newSchedule = Schedule().initalize()
        for idx in range(0, len(newSchedule.classes)):
            if random.random() < 0.2 :sched.classes[idx] = newSchedule.classes[idx]
        return newSchedule

In [29]:
class disp:
    def printData(self, population):
        df = pd.DataFrame(columns=['Schedule no.', 'Fitness', 'Total conflicts'])
        df_1 = pd.DataFrame(columns=['Class', 'Section', 'Course','Room','Duration - Day',''])
        Schedu = population.schedule
        classe = Schedu[0].classes
        for i in range(0, len(Schedu)):
            df.loc[i] = [str(i), round(Schedu[i].fitnessValue(), 3), Schedu[i].total_conf]
        for i in range(0, len(classe)):
            df_1.loc[i] = [str(i), classe[i].section, classe[i].course, classe[i].classroom.code, (classe[i].duration.time,classe[i].duration.day), classe[i].teacher ]
        print(df)
        print(df_1)

In [32]:
info = Information()
def work():
    pop = Population(total_population)
    pop.schedule.sort(key=lambda x: x.fitnessValue(), reverse=True)
    d = disp()
    print("-------Iteration------",0)
    d.printData(pop)
    geneticAlgorithm, g = GenAlgo(pop), 0
    while True:
        if pop.schedule[0].fitnessValue() == 1.0:break
        g += 1
        pop = geneticAlgorithm.start()
        pop.schedule.sort(key=lambda x: x.fitnessValue(), reverse=True)
        print("-------Iteration------",g)
        d.printData(pop)
work()

no class on friday between 1:00 t0 2:00
no class after 5 adn before 8:30
no class after 5 adn before 8:30
no class on friday between 1:00 t0 2:00
no class on sunday and saturday
teacher consective classes Abdul Jalil 6.0 monday
no class on friday between 1:00 t0 2:00
no class on friday between 1:00 t0 2:00
no class after 5 adn before 8:30
no class after 5 adn before 8:30
no class on friday between 1:00 t0 2:00
no class on sunday and saturday
teacher consective classes Abdul Jalil 7.0 monday
no class after 5 adn before 8:30
no class after 5 adn before 8:30
no class on friday between 1:00 t0 2:00
no class on sunday and saturday
teacher consective classes Abdul Jalil 5.0 monday
classroom, teacher, section conflict
classroom, teacher, section conflict
no class on friday between 1:00 t0 2:00
classroom, teacher, section conflict
classroom, teacher, section conflict
no class after 5 adn before 8:30
no class after 5 adn before 8:30
no class on friday between 1:00 t0 2:00
no class on sunday and